In [1]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import altair as alt
import seaborn as sns
import re

In [2]:
#Make sure numpy version is < 1.20
np.version.version

'1.18.5'

In [224]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [225]:
from gensim.models.word2vec import Word2Vec
from tqdm.notebook import tqdm
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mryua\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mryua\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mryua\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [226]:
RANDOM_SEED=694

Training Dataset

In [227]:
train_path = 'Data/WikiLarge_Train.csv'
df = pd.read_csv(train_path, skiprows=0, skipfooter=0, engine='python')
df

,original_text,label
0,There is manuscript evidence that Austen conti...,1
1,"In a remarkable comparative analysis , Mandaea...",1
2,"Before Persephone was released to Hermes , who...",1
3,Cogeneration plants are commonly found in dist...,1
4,"Geneva -LRB- , ; , ; , ; ; -RRB- is the second...",1
...,...,...
416763,A Duke Nukem 3D version has been sold for Xbox...,0
416764,"However , it is becoming replaced as a method ...",0
416765,There are hand gestures in both Hindu and Budd...,0
416766,"If it is necessary to use colors , try to choo...",0


In [228]:
len(df[df['label']==1])/len(df) # the dataset label is well balanced 

0.5

In [229]:
df.iloc[50]['original_text']

'He studied in Armenia and Istanbul , then at Wisconsin University which he finished in 1915 .'

In [230]:
df['original_text'].apply(lambda x: len(x)).mean()
# This means all texts are considered short text, which allows us to use dense representations, 
# as dense representations work well with short text.
# Gensim.KeyedVectors.load('assets/wikipedia.100.word-vecs.kv')??? How to generate and use this???
# Maybe we should train word2vec model on the entire corpus. Just training data? TOP 100 word-vectors(features)
# Alternatively we could use bag-of-words model, which is term-document matrix representation, having much more features

117.921906192414

In [231]:
X = df['original_text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Testing Dataset

In [232]:
test_path = 'Data/WikiLarge_Test.csv'
test_df = pd.read_csv(test_path, skiprows=0, skipfooter=0, engine='python')
test_df

,id,original_text,label
0,0,-2011,NaN
1,1,-2011,NaN
2,2,-2000,NaN
3,3,-1997,NaN
4,4,1.636,NaN
...,...,...,...
119087,119087,#NAME?,NaN
119088,119088,#NAME?,NaN
119089,119089,#NAME?,NaN
119090,119090,#NAME?,NaN


In [233]:
test_df.iloc[10000]

id                                                           10000
original_text    An atheist would say that this argument proves...
label                                                          NaN
Name: 10000, dtype: object

Sample Submission

In [234]:
samplesubmission_path = 'Data/sampleSubmission.csv'
samplesubmission_df = pd.read_csv(samplesubmission_path, skiprows=0, skipfooter=0, engine='python')
samplesubmission_df

,id,label
0,0,0
1,1,0
2,2,1
3,3,1
4,4,0
...,...,...
119087,119087,0
119088,119088,1
119089,119089,1
119090,119090,1


To conclude, the dataframes we are working with are:

dalechall_df, concreteness_df, aoawords_df, train_df, test_df, samplesubmission_df

# 1. Data Preprocessing

In [244]:
vectorizer = TfidfVectorizer(min_df=10,stop_words='english',ngram_range=(1,2))
X_train_transform = vectorizer.fit_transform(X_train)
X_test_transform  = vectorizer.transform(X_test)

In [245]:
X_train_transform

<333414x57773 sparse matrix of type '<class 'numpy.float64'>'
	with 4071111 stored elements in Compressed Sparse Row format>

## Word2Vec Model

In [246]:
len(stopwords.words('english'))

179

In [247]:
len(set(stopwords.words('english')))

179

In [262]:
#set(stopwords.words('english'))

== dale_chall.txt ==

This is the Dale Chall 3000 Word List, which is one definition of words that are considered "basic" English.

A summary is at https://www.readabilityformulas.com/articles/dale-chall-readability-word-list.php

In [249]:
#Basic english words
dalechall_path = 'Data/dale_chall.txt'
dale_chall = pd.read_csv(dalechall_path,delimiter='\t',header=None,names=['word'])
dale = set(dale_chall['word'].values)

In [250]:
len(dale)

2946

### The 2946 words in dale can be combined with the nltk stopwords.
### We could maybe assign an arbitrary score to each dale_chall word. - for reference only

In [251]:
stopWords = set(stopwords.words('english')) | dale

In [252]:
len(stopWords)

2986

In [253]:
X_train

304501    1979-80 Buffalo Sabres NHL 32 1880 74 1 4 2.36...
162313    Diseases Lentils in culture Lentils are mentio...
336845    Railroads , like the Lehigh Valley Railroad , ...
150625    An example of this would be an individual anim...
40240     Both the Matanuska and Susitna Rivers have maj...
                                ...                        
259178    After the Germans invaded Norway in April 1940...
365838    July 28 - Henry Bennet , 1st Earl of Arlington...
131932    Pancake restaurants are popular family restaur...
146867                                 A cycling domestique
121958    David Boreanaz 's first paid acting appearance...
Name: original_text, Length: 333414, dtype: object

In [254]:
X_train[304501]

'1979-80 Buffalo Sabres NHL 32 1880 74 1 4 2.36 20 8 4 0 0.000'

In [255]:
gensim.utils.simple_preprocess(X_train[304501])

['buffalo', 'sabres', 'nhl']

In [256]:
#gensim.parsing.preprocessing.STOPWORDS
#stopWords

In [263]:
tokenized_text_train=[]
tokenized_text_test=[]
stopWords = set(stopwords.words('english')) | dale
# This cell will run 4 minutes
import gensim
from nltk.stem.porter import *
def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    #Un-hash next line to use stemming
    #return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
    #Un-hash next line to NOT use stemming
    return WordNetLemmatizer().lemmatize(text, pos='v')

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            #Un-hash next line to use lemmatization/stemming
            result.append(lemmatize_stemming(token))
            #Un-hash next line to NOT use lemmatization/stemming
            #result.append(token)
            
    return result

tokenized_text_train = [preprocess(text) for text in X_train]
tokenized_text_test=[preprocess(text) for text in X_test]

#for text in tqdm(X_train):
#    tokens_in_text = word_tokenize(text)
#    tokens_in_text = [word for word in tokens_in_text if word.lower() not in stopWords]
#    tokenized_text_train.append(tokens_in_text)
    
#for text in tqdm(X_test):
#    tokens_in_text = word_tokenize(text)
#    tokens_in_text = [word for word in tokens_in_text if word.lower() not in stopWords]
#    tokenized_text_test.append(tokens_in_text)

In [270]:
#tokenized_text_train

In [271]:
model = Word2Vec(vector_size=100,window=2,min_count=100,seed= RANDOM_SEED,workers=4)
model.build_vocab(tokenized_text_train)
model.train(tokenized_text_train,total_examples=model.corpus_count,epochs=model.epochs)

(11285460, 15511955)

In [272]:
word_vectors = model.wv

In [297]:
word_vectors.vocab

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

In [298]:
word_dict = word_vectors.key_to_index

In [299]:
word_dict

{'state': 0,
 'bear': 1,
 'france': 2,
 'city': 3,
 'know': 4,
 'unite': 5,
 'department': 6,
 'play': 7,
 'region': 8,
 'commune': 9,
 'time': 10,
 'north': 11,
 'world': 12,
 'football': 13,
 'american': 14,
 'call': 15,
 'include': 16,
 'people': 17,
 'south': 18,
 'game': 19,
 'work': 20,
 'team': 21,
 'national': 22,
 'form': 23,
 'county': 24,
 'player': 25,
 'district': 26,
 'write': 27,
 'release': 28,
 'year': 29,
 'years': 30,
 'english': 31,
 'name': 32,
 'second': 33,
 'live': 34,
 'locate': 35,
 'number': 36,
 'west': 37,
 'music': 38,
 'band': 39,
 'film': 40,
 'area': 41,
 'group': 42,
 'record': 43,
 'town': 44,
 'university': 45,
 'series': 46,
 'later': 47,
 'september': 48,
 'season': 49,
 'build': 50,
 'british': 51,
 'january': 52,
 'base': 53,
 'march': 54,
 'calais': 55,
 'east': 56,
 'album': 57,
 'school': 58,
 'begin': 59,
 'german': 60,
 'october': 61,
 'start': 62,
 'july': 63,
 'league': 64,
 'take': 65,
 'august': 66,
 'book': 67,
 'river': 68,
 'june': 69

In [348]:
word_vectors[0] == word_vectors['state']

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [301]:
words_in_vector = word_vectors.index_to_key
len(words_in_vector)

4294

# Word's Difficulty Considered

== Concreteness_ratings_Brysbaert_et_al_BRM.txt ==

This file contains concreteness ratings for 40 thousand English lemma words gathered via Amazon Mechanical Turk. The ratings come from a larger list of 63 thousand words and represent all English words known to 85% of the raters.

The file contains eight columns:
1. The word
2. Whether it is a single word or a two-word expression 
3. The mean concreteness rating
4. The standard deviation of the concreteness ratings
5. The number of persons indicating they did not know the word
6. The total number of persons who rated the word
7. Percentage participants who knew the word
8. The SUBTLEX-US frequency count (on a total of 51 million; Brysbaert & New, 2009) 
9. The dominant part-of-speech usage

Original source: http://crr.ugent.be/archives/1330

Brysbaert, M., Warriner, A.B., & Kuperman, V. (2014). Concreteness ratings for 40 thousand generally known English word lemmas. Behavior Research Methods, 46, 904-911.
http://crr.ugent.be/papers/Brysbaert_Warriner_Kuperman_BRM_Concreteness_ratings.pdf

In [302]:
#Concreteness rating - the higher Conc.M, the easier the word is.
concreteness_path = 'Data/Concreteness_ratings_Brysbaert_et_al_BRM.txt'
concrete_df = pd.read_csv(concreteness_path,delimiter='\t', keep_default_na=False)
concreteset=(concrete_df['Word'].values)

In [303]:
concrete_df

,Word,Bigram,Conc.M,Conc.SD,Unknown,Total,Percent_known,SUBTLEX,Dom_Pos
0,roadsweeper,0,4.85,0.37,1,27,0.96,0,0
1,traindriver,0,4.54,0.71,3,29,0.90,0,0
2,tush,0,4.45,1.01,3,25,0.88,66,0
3,hairdress,0,3.93,1.28,0,29,1.00,1,0
4,pharmaceutics,0,3.77,1.41,4,26,0.85,0,0
...,...,...,...,...,...,...,...,...,...
39949,unenvied,0,1.21,0.62,1,30,0.97,0,#N/A
39950,agnostically,0,1.20,0.50,2,27,0.93,0,#N/A
39951,conceptualistic,0,1.18,0.50,4,26,0.85,0,#N/A
39952,conventionalism,0,1.18,0.48,1,29,0.97,0,#N/A


In [304]:
concrete_df.Bigram.value_counts()

0    37058
1     2896
Name: Bigram, dtype: int64

In [305]:
concrete_df[concrete_df.Bigram==1]

,Word,Bigram,Conc.M,Conc.SD,Unknown,Total,Percent_known,SUBTLEX,Dom_Pos
28707,baking soda,1,5.00,0.00,0,30,1.00,0,#N/A
28709,baseball bat,1,5.00,0.00,0,29,1.00,0,#N/A
28710,bath towel,1,5.00,0.00,0,29,1.00,0,#N/A
28711,beach ball,1,5.00,0.00,0,28,1.00,0,#N/A
28712,bed sheet,1,5.00,0.00,0,28,1.00,0,#N/A
...,...,...,...,...,...,...,...,...,...
39619,tantamount to,1,1.52,0.85,4,27,0.85,0,#N/A
39857,chance on,1,1.38,0.75,2,28,0.93,0,#N/A
39871,free rein,1,1.37,0.63,2,29,0.93,0,#N/A
39899,by chance,1,1.34,0.72,1,30,0.97,0,#N/A


In [306]:
#There is no Nan value in Conc.M column
concrete_df[concrete_df['Conc.M'].isna()]

,Word,Bigram,Conc.M,Conc.SD,Unknown,Total,Percent_known,SUBTLEX,Dom_Pos


### Are we gonna consider bigrams in this dataset, given it's only a small fraction ~ 8% in size?

In [307]:
np.min(concrete_df['Conc.M'])

1.04

In [308]:
np.max(concrete_df['Conc.M'])

5.0

### Concreteness values range from 1 - 5, we could possible use the inverse value of concreteness to scale it to a 0-1 range and give easier words less weight.

In [309]:
concrete_words = list(concrete_df['Word'].values)

In [310]:
len(concrete_words)

39954

In [311]:
concrete_complement = [word for word in words_in_vector if word not in concrete_words]

In [312]:
len(concrete_complement)

1187

In [313]:
concrete_complement[:20]

['france',
 'american',
 'years',
 'english',
 'british',
 'calais',
 'german',
 'england',
 'french',
 'london',
 'largest',
 'ndash',
 'germany',
 'japanese',
 'york',
 'australia',
 'america',
 'members',
 'nord',
 'italian']

In [314]:
concrete_intersect = [word for word in words_in_vector if word in concrete_list]

In [315]:
len(concrete_intersect)

3107

In [316]:
concrete_intersect[0]

'state'

In [318]:
word_vectors['state']

array([ 1.4483625e+00, -2.2964539e-02,  1.8962008e-03,  5.4766673e-01,
       -2.6734391e-01, -6.7329913e-01,  3.6649238e-02,  7.5110155e-01,
        2.6593966e-02,  3.6272058e-01,  1.8063091e-01, -8.1770726e-02,
       -1.2644223e+00, -1.1407950e+00,  1.3186027e+00,  1.5527658e-01,
       -4.0354431e-01,  3.4420010e-01,  7.5570363e-01,  1.0251225e+00,
       -3.3162057e-01,  5.4691845e-01,  2.8648919e-01,  4.1449976e-01,
        4.7424209e-01, -8.8924634e-01,  1.3474070e+00, -5.9115124e-01,
       -2.3832204e-02,  2.1102323e-01,  1.1408305e+00, -2.3086752e-01,
        3.8275310e-01,  6.7911857e-01, -1.9161204e-01,  2.0487516e-01,
        4.2912993e-01, -1.9336376e-01,  7.2722673e-02, -1.0755360e+00,
       -1.4390081e+00, -5.9247285e-01,  3.6843863e-01, -2.0793953e+00,
        5.6404817e-01, -1.5470855e-01,  8.4685934e-01,  7.3713817e-02,
       -1.0202421e+00,  3.3701974e-01, -1.2711020e+00,  7.2744918e-01,
       -7.4224278e-02,  1.1051776e+00, -1.0541314e+00, -5.4030412e-01,
      

In [321]:
concrete_df[concrete_df['Word']=='state']['Conc.M'].values

array([3.52])

In [320]:
for word in concrete_intersect:
    word_vectors[word] = word_vectors[word] * 1/concrete_df[concrete_df['Word']==word]['Conc.M'].values

== AoA_51715_words.csv ==

This file contains "Age of Acquisition" (AoA) estimates for about 51k English words, which refers to the approximate age (in years) when a word was learned. Early words, being more basic, have lower average AoA.

The main columns you will be interested in are "Word" and "AoA_Kup_lem". But the others may be useful too.

The file contains these columns:

Word :: The word in question
Alternative.spelling :: if the Word may be spelled frequently in another form	
Freq_pm	:: Freq of the Word in general English (larger -> more common)
Dom_PoS_SUBTLEX	:: Dominant part of speech in general usage
Nletters :: number of letters 
Nphon :: number of phonemes
Nsyll :: number of syllables
Lemma_highest_PoS :: the "lemmatized" or "root" form of the word (in the dominant part of speech. e.g. The root form of the verb "abates" is "abate".
AoA_Kup	:: The AoA from a previous study by Kuperman et al.
Perc_known :: Percent of people who knew the word in the Kuperman et al. study
AoA_Kup_lem :: Estimated AoA based on Kuperman et al. study lemmatized words. THIS IS THE MAIN COLUMN OF INTEREST.
Perc_known_lem	:: Estimated percentage of people who would know this form of the word in the Kuperman study.
AoA_Bird_lem :: AoA reported in previous study by Bird (2001) 
AoA_Bristol_lem	:: AoA reported in previous study from Bristol Univ. (2006)
AoA_Cort_lem :: AoA reported in previous study by Cortese & Khanna (2008)
AoA_Schock :: AoA reported in previous study by Schock (2012)

Original source : http://crr.ugent.be/archives/806

In [322]:
#AoA
#Perc_known_lem, AoA_Kup_lem
aoawords_path = 'Data/AoA_51715_words.csv'
AoA = pd.read_csv(aoawords_path,encoding = 'unicode_escape')
AoA_set = set(AoA['Word'].values)
AoA.head(5)

,Word,Alternative.spelling,Freq_pm,Dom_PoS_SUBTLEX,Nletters,Nphon,Nsyll,Lemma_highest_PoS,AoA_Kup,Perc_known,AoA_Kup_lem,Perc_known_lem,AoA_Bird_lem,AoA_Bristol_lem,AoA_Cort_lem,AoA_Schock
0,a,a,20415.27,Article,1,1,1,a,2.89,1.00,2.89,1.00,3.16,NaN,NaN,NaN
1,aardvark,aardvark,0.41,Noun,8,7,2,aardvark,9.89,1.00,9.89,1.00,NaN,NaN,NaN,NaN
2,abacus,abacus,0.24,Noun,6,6,3,abacus,8.69,0.65,8.69,0.65,NaN,NaN,NaN,NaN
3,abacuses,abacuses,0.02,Noun,8,9,4,abacus,NaN,NaN,8.69,0.65,NaN,NaN,NaN,NaN
4,abalone,abalone,0.51,Verb,7,7,4,abalone,12.23,0.72,12.23,0.72,NaN,NaN,NaN,NaN


In [323]:
len(AoA)

51715

In [324]:
AoA.AoA_Kup_lem.min()

1.58

In [325]:
AoA.AoA_Kup_lem.max()

25.0

In [326]:
AoA.sort_values(['AoA_Kup_lem'], ascending=False)

,Word,Alternative.spelling,Freq_pm,Dom_PoS_SUBTLEX,Nletters,Nphon,Nsyll,Lemma_highest_PoS,AoA_Kup,Perc_known,AoA_Kup_lem,Perc_known_lem,AoA_Bird_lem,AoA_Bristol_lem,AoA_Cort_lem,AoA_Schock
14878,eisteddfod,eisteddfod,NaN,NaN,10,8,3,eisteddfod,25.0,0.05,25.0,0.05,NaN,NaN,NaN,NaN
2084,architrave,architrave,0.04,Noun,10,8,3,architrave,21.0,0.05,21.0,0.05,NaN,NaN,NaN,NaN
6274,calceolaria,calceolaria,0.02,Noun,11,11,6,calceolaria,21.0,0.11,21.0,0.11,NaN,NaN,NaN,NaN
32931,penury,penury,0.02,Noun,6,7,3,penury,20.6,0.28,20.6,0.28,NaN,NaN,NaN,NaN
25243,kendo,kendo,0.37,Noun,5,5,2,kendo,20.5,0.11,20.5,0.11,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38932,rogation,rogation,NaN,NaN,8,7,3,rogation,NaN,0.00,NaN,0.00,NaN,NaN,NaN,NaN
42089,smilax,smilax,NaN,NaN,6,7,2,smilax,NaN,0.00,NaN,0.00,NaN,NaN,NaN,NaN
46368,thulium,thulium,NaN,NaN,7,6,3,thulium,NaN,0.00,NaN,0.00,NaN,NaN,NaN,NaN
50862,wickiup,wickiup,0.27,Noun,7,6,3,wickiup,NaN,0.00,NaN,0.00,NaN,NaN,NaN,NaN


In [327]:
len(AoA[AoA['AoA_Kup_lem'].isna()])

20

In [328]:
AoA[AoA['AoA_Kup_lem'].isna()]

,Word,Alternative.spelling,Freq_pm,Dom_PoS_SUBTLEX,Nletters,Nphon,Nsyll,Lemma_highest_PoS,AoA_Kup,Perc_known,AoA_Kup_lem,Perc_known_lem,AoA_Bird_lem,AoA_Bristol_lem,AoA_Cort_lem,AoA_Schock
442,actinium,actinium,NaN,NaN,8,8,4,actinium,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
1322,ambuscade,ambuscade,NaN,NaN,9,8,3,ambuscade,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
2306,ashlar,ashlar,NaN,NaN,6,5,2,ashlar,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
5095,bosky,bosky,NaN,NaN,5,4,2,bosky,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
6404,canaille,canaille,NaN,NaN,8,5,2,canaille,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
9004,compeer,compeer,NaN,NaN,7,6,3,compeer,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
9005,compeers,compeers,0.02,Noun,8,7,3,compeer,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
16000,europium,europium,NaN,NaN,8,8,4,europium,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
19065,gallimaufry,gallimaufry,NaN,NaN,11,9,4,gallimaufry,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
22498,hutment,hutment,NaN,NaN,7,7,2,hutment,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN


In [329]:
# We are going to impute all Nan values in AoA_Kup_lem as the max AoA value 25, as they appear to be hard words.
AoA['AoA_Kup_lem'].fillna(value=AoA['AoA_Kup_lem'].max(), inplace=True)

In [330]:
AoA.sort_values(['AoA_Kup_lem'], ascending=False)

,Word,Alternative.spelling,Freq_pm,Dom_PoS_SUBTLEX,Nletters,Nphon,Nsyll,Lemma_highest_PoS,AoA_Kup,Perc_known,AoA_Kup_lem,Perc_known_lem,AoA_Bird_lem,AoA_Bristol_lem,AoA_Cort_lem,AoA_Schock
2306,ashlar,ashlar,NaN,NaN,6,5,2,ashlar,NaN,0.00,25.00,0.00,NaN,NaN,NaN,NaN
38932,rogation,rogation,NaN,NaN,8,7,3,rogation,NaN,0.00,25.00,0.00,NaN,NaN,NaN,NaN
46368,thulium,thulium,NaN,NaN,7,6,3,thulium,NaN,0.00,25.00,0.00,NaN,NaN,NaN,NaN
14878,eisteddfod,eisteddfod,NaN,NaN,10,8,3,eisteddfod,25.00,0.05,25.00,0.05,NaN,NaN,NaN,NaN
5095,bosky,bosky,NaN,NaN,5,4,2,bosky,NaN,0.00,25.00,0.00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27395,mamma,mamma,3.02,Noun,5,4,2,mama,NaN,NaN,1.89,1.00,NaN,NaN,NaN,NaN
27393,mamas,mamas,0.71,Noun,5,5,2,mama,NaN,NaN,1.89,1.00,NaN,NaN,NaN,NaN
27392,mama,mama,103.71,Noun,4,4,2,mama,1.89,1.00,1.89,1.00,NaN,NaN,NaN,NaN
29050,mommas,mommas,0.10,Noun,6,5,2,momma,NaN,NaN,1.58,1.00,NaN,NaN,NaN,NaN


### AoA values range from 0 - 25, which means the smaller the AoA value, the easier the word is. We could possibly use the AoA value to give easier words less weight.

In [331]:
aoa_words = list(AoA['Word'].values)

In [332]:
len(aoa_words)

51715

In [333]:
aoa_complement = [word for word in words_in_vector if word not in aoa_words]
aoa_intersect = [word for word in words_in_vector if word in aoa_words]

In [334]:
len(aoa_complement)

988

In [335]:
aoa_complement

['france',
 'american',
 'english',
 'september',
 'british',
 'calais',
 'german',
 'october',
 'july',
 'june',
 'april',
 'england',
 'february',
 'french',
 'london',
 'john',
 'ndash',
 'germany',
 'usually',
 'japanese',
 'york',
 'australia',
 'america',
 'nord',
 'italian',
 'aisne',
 'europe',
 'pakistan',
 'spanish',
 'greek',
 'european',
 'india',
 'calvados',
 'william',
 'california',
 'basse',
 'normandie',
 'switzerland',
 'george',
 'canton',
 'commonly',
 'canada',
 'charles',
 'china',
 'australian',
 'italy',
 'loire',
 'christian',
 'henry',
 'gironde',
 'scotland',
 'canadian',
 'ireland',
 'africa',
 'picardie',
 'paul',
 'florida',
 'aquitaine',
 'brazilian',
 'generally',
 'atlantic',
 'latin',
 'david',
 'robert',
 'chinese',
 'russian',
 'asia',
 'paris',
 'scottish',
 'indian',
 'dutch',
 'berlin',
 'officially',
 'richard',
 'lower',
 'spain',
 'iowa',
 'especially',
 'disney',
 'michael',
 'thomas',
 'peter',
 'wales',
 'britain',
 'wikipedia',
 'approxima

In [336]:
len(aoa_intersect)

3306

In [337]:
aoa_intersect[:20]

['state',
 'bear',
 'city',
 'know',
 'unite',
 'department',
 'play',
 'region',
 'commune',
 'time',
 'north',
 'world',
 'football',
 'call',
 'include',
 'people',
 'south',
 'game',
 'work',
 'team']

In [338]:
len([word for word in aoa_intersect if word in concrete_intersect])

2965

In [339]:
for word in aoa_intersect:
    word_vectors[word] = word_vectors[word] * AoA[AoA['Word']==word]['AoA_Kup_lem'].values/25

In [341]:
word_vectors['state']

array([ 1.05170861e-01, -1.66753866e-03,  1.37690033e-04,  3.97680737e-02,
       -1.94128137e-02, -4.88906987e-02,  2.66123447e-03,  5.45402169e-02,
        1.93108455e-03,  2.63384599e-02,  1.31162666e-02, -5.93766989e-03,
       -9.18143019e-02, -8.28372762e-02,  9.57485363e-02,  1.12751964e-02,
       -2.93028187e-02,  2.49936208e-02,  5.48743866e-02,  7.44378790e-02,
       -2.40801759e-02,  3.97137366e-02,  2.08030213e-02,  3.00983358e-02,
        3.44364420e-02, -6.45714104e-02,  9.78401154e-02, -4.29256409e-02,
       -1.73054298e-03,  1.53231639e-02,  8.28398466e-02, -1.67641304e-02,
        2.77930945e-02,  4.93132696e-02, -1.39136473e-02,  1.48767298e-02,
        3.11606843e-02, -1.40408454e-02,  5.28065767e-03, -7.80985802e-02,
       -1.04491614e-01, -4.30216081e-02,  2.67536677e-02, -1.50992453e-01,
        4.09575887e-02, -1.12339500e-02,  6.14935383e-02,  5.35262842e-03,
       -7.40834847e-02,  2.44722292e-02, -9.22993347e-02,  5.28227314e-02,
       -5.38969506e-03,  

In [342]:
def generate_dense_features(tokenized_text,word_vectors):
    dense_list=[]
    words=[]
    for _ in tokenized_text: 
        words =[word for word in _ if word in word_vectors.key_to_index]
        
        if len(words) >0:
            dense_list.append(np.mean(word_vectors[words],axis=0))
            
        else: 
            dense_list.append(np.zeros(word_vectors.vector_size))
            
    return np.array(dense_list)

In [343]:
X_train_wv = generate_dense_features(tokenized_text_train,word_vectors)
X_test_wv = generate_dense_features(tokenized_text_test,word_vectors)

In [344]:
X_train_wv.shape

(333414, 100)

In [345]:
lr_wv = LogisticRegression(random_state=RANDOM_SEED,max_iter=1000).fit(X_train_wv,y_train)

In [346]:
accuracy_score(y_test,lr_wv.predict(X_test_wv))

0.561676704177364

## Bag of Words Model

In [148]:
def dummy_fun(doc):
    return doc
vectorizer = TfidfVectorizer(analyzer='word',tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=r'(?u)\b\w\w+__\([\w\s]*\)')
X_train_transform = vectorizer.fit_transform(tokenized_text_train)
X_test_transform  = vectorizer.transform(tokenized_text_test)

In [149]:
len(vectorizer.get_feature_names())

120723

In [150]:
vectorizer.get_feature_names()

['a_nx',
 'aabout',
 'aabye',
 'aach',
 'aachen',
 'aafc',
 'aage',
 'aaiil',
 'aalborg',
 'aalen',
 'aaliyah',
 'aaliyahs',
 'aall',
 'aalst',
 'aalten',
 'aalter',
 'aalto',
 'aames',
 'aamir',
 'aang',
 'aangã',
 'aapep',
 'aarau',
 'aarberg',
 'aarburg',
 'aarc',
 'aarde',
 'aardman',
 'aardsma',
 'aardvark',
 'aardvarks',
 'aare',
 'aargau',
 'aargauer',
 'aarhus',
 'aaron',
 'aaroni',
 'aarons',
 'aarre',
 'aarschot',
 'aarseth',
 'aartselaar',
 'aarwangen',
 'aasen',
 'aashurah',
 'aast',
 'aastana',
 'aave',
 'ababa',
 'ababba',
 'ababda',
 'abac',
 'abacada',
 'abaci',
 'aback',
 'abacus',
 'abacuses',
 'abad',
 'abagnale',
 'abahutu',
 'abaj',
 'abajo',
 'abakan',
 'abakanskoye',
 'abal',
 'abalo',
 'abalone',
 'abando',
 'abandon',
 'abandonded',
 'abandoned',
 'abandoning',
 'abandonment',
 'abandons',
 'abarat',
 'abassi',
 'abated',
 'abating',
 'abattoirs',
 'abatutsi',
 'abauzit',
 'abavo',
 'abazhou',
 'abaãºj',
 'abba',
 'abbado',
 'abbadon',
 'abbados',
 'abbandando'

In [151]:
X_train_transform

<333414x120723 sparse matrix of type '<class 'numpy.float64'>'
	with 2896838 stored elements in Compressed Sparse Row format>

In [152]:
lr_bow = LogisticRegression(random_state=RANDOM_SEED,max_iter=1000).fit(X_train_transform,y_train)

In [153]:
accuracy_score(y_test,lr_bow.predict(X_test_transform))

0.6584686997624589

In [181]:
model_word = set(word_vectors.index_to_key) #around 6k words in the Word2Vec model

In [182]:
len(model_word)

4031

In [183]:
len(model_word.intersection(concreteset))

2048

In [184]:
word_vectors['live']

array([-0.03942373,  1.7966397 , -0.14200447, -0.3486227 ,  0.03800169,
       -0.131528  , -0.35020527, -0.41108254,  1.6974189 ,  0.05236067,
       -0.00799712,  0.7041458 ,  0.03987895, -0.15628862, -0.01195999,
       -0.5276748 , -0.28911754, -0.59118384, -0.29640502, -0.05531655,
        0.05208854,  1.2201724 ,  0.05189531, -1.7594199 ,  0.1870284 ,
        0.26337144,  0.03394307,  0.18608569, -0.55623055, -0.96030605,
       -0.70980805,  0.70162207, -1.1859473 , -0.61812186, -0.6310784 ,
       -0.5761929 , -0.23991399,  0.7104913 ,  1.6777844 , -0.02619001,
        1.5803056 , -0.43163323, -0.37865758, -0.46269822,  1.1120914 ,
       -0.24402891,  0.1181486 , -1.052571  , -0.37906688,  1.0918018 ,
        0.18887055,  0.6383479 ,  0.99249554,  1.1353667 ,  0.57656294,
        0.21096966,  0.22929311, -0.43283883,  0.50271827,  0.54883325,
        0.30768827,  0.03781867, -0.242916  , -1.5159138 , -0.63193476,
        0.3270656 ,  1.6821493 , -0.28726235,  0.28426522,  0.38

In [185]:
lemmatizer = WordNetLemmatizer()
word_list = []
for word in model_word: 
    word_list.append((word,lemmatizer.lemmatize(word.lower())))
df = pd.DataFrame(word_list,columns=['Original','word'])
df = df.merge(AoA,left_on='word',right_on='Word',how='left')
df = df[['Original','word','Perc_known','AoA_Kup_lem']]
word_not_matched = set(df[df['Perc_known'].isnull()].word.values)

for i in range(len(df)):   
    if df['word'][i][0] in set(('0','1','2','3','4','5','6','7','8','9')) or len(df['word'][i])==1:
        df['AoA_Kup_lem'][i] = 3
mean_value = df['AoA_Kup_lem'].mean()
df['AoA_Kup_lem'].fillna(value=mean_value,inplace=True)

In [186]:
#df.loc[df['Original']==['troops','weapons']]
df[df['Original'].isin(['troops','weapon'])]

,Original,word,Perc_known,AoA_Kup_lem
3094,weapon,weapon,1.0,6.95


In [187]:
def generate_perc_known(tokenized_text,df):
    avg_perc_know=None
    perc_know_list=[]
    for _ in tokenized_text: 
        words =[word for word in _ if word in word_vectors.key_to_index]
        
        if len(words) >0:
            avg_perc_know = np.mean(df[df['Original'].isin(words)]['AoA_Kup_lem'])
            perc_know_list.append(avg_perc_know)
        else: 
            
            perc_know_list.append(0)
            
    return perc_know_list

In [188]:
df_train = pd.DataFrame(X_train_wv)
df_train['year'] = generate_perc_known(tokenized_text_train,df)

In [189]:
df_test = pd.DataFrame(X_test_wv)
df_test['year'] = generate_perc_known(tokenized_text_test,df)

In [190]:
df_test

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,year
0,0.163066,0.066503,0.007967,-0.368197,-0.475971,0.174799,-0.226500,0.288741,-0.101118,0.251682,...,0.343301,0.449110,-0.301583,-0.318929,-0.027628,-0.003120,0.640888,0.395134,-0.211103,7.319698
1,0.098105,-0.697004,-0.067849,0.073167,0.001977,-0.519177,-0.064798,-0.384014,0.359658,-0.080730,...,0.100243,-0.152842,0.018108,-0.616458,0.208961,0.239500,-0.078117,0.907243,0.644744,8.900953
2,0.608009,-0.270855,-0.351858,-1.324698,0.509448,0.466696,-0.869674,0.316894,-0.832663,0.482958,...,-0.804097,-1.260673,-0.484280,-1.026836,-0.381989,0.006748,0.651532,0.502151,-1.543706,7.385000
3,-0.231419,-0.460309,-0.321846,-0.401228,-1.299778,-0.461486,0.002258,-0.175611,0.296010,0.373852,...,-0.068769,0.134842,0.026607,0.200088,0.376173,0.175164,-0.239718,0.463941,-0.541556,8.971588
4,-0.155188,0.110082,0.749716,-0.211680,-0.294006,-0.928232,0.095029,0.326077,0.020296,0.458989,...,-0.496064,0.562254,-0.161042,-0.556670,-0.152797,0.216482,-0.109737,1.134926,-0.073294,7.939948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83349,-0.212178,-0.577913,0.233901,-0.283749,-0.250686,-0.740940,-0.073741,0.163121,-0.268991,0.569778,...,-0.062439,0.181861,-0.294118,0.674152,-0.312687,-0.416863,0.006465,0.296494,0.144787,7.846061
83350,0.083994,-0.119798,0.014636,-0.046240,-0.176528,-0.371178,-0.049741,-0.063575,0.069346,0.097987,...,-0.106163,0.598239,-0.423099,-0.277646,0.249423,0.238795,-0.084238,0.325800,-0.371009,7.653076
83351,-0.027579,-0.583053,-0.212853,0.064448,-0.001676,-0.386104,-0.194504,0.125628,0.087920,0.013556,...,-0.087076,0.200042,0.022237,0.865286,0.345294,0.206362,-0.050420,0.287032,-0.024188,6.618984
83352,0.150752,-0.344787,0.016055,-0.438976,0.105028,-0.072180,-0.229091,0.010541,-0.065317,0.162644,...,-0.090400,-0.352687,-0.262663,-0.028436,0.180446,-0.053098,0.068634,-0.034959,0.074879,7.009195


In [191]:
lr = LogisticRegression(random_state=RANDOM_SEED,max_iter=1000).fit(df_train,y_train)

In [192]:
accuracy_score(y_test,lr.predict(df_test))

0.58372723564556

# 2. Supervised Learning

## Random Classifier

In [64]:
dummy_bow = DummyClassifier(strategy='uniform',random_state=RANDOM_SEED).fit(X_train_transform,y_train)

In [65]:
accuracy_score(y_test, dummy_bow.predict(X_test_transform))

0.5011277203253593

In [66]:
dummy_wv = DummyClassifier(strategy='uniform',random_state=RANDOM_SEED).fit(X_train_wv,y_train)

In [67]:
accuracy_score(y_test,dummy_wv.predict(X_test_wv))

0.5011277203253593

## Logistic Regression Classifier

In [68]:
lr_bow = LogisticRegression(random_state=RANDOM_SEED,max_iter=1000).fit(X_train_transform,y_train)

In [69]:
accuracy_score(y_test,lr_bow.predict(X_test_transform))

0.6465916452719725

In [70]:
lr_wv = LogisticRegression(random_state=RANDOM_SEED,max_iter=1000).fit(X_train_wv,y_train)

In [71]:
accuracy_score(y_test,lr_wv.predict(X_test_wv))

0.5640041269765098

## Random Forest Classifier

In [227]:
rf_bow = RandomForestClassifier(n_estimators=500,max_depth=5,random_state=RANDOM_SEED).fit(X_train_transform,y_train)

In [228]:
accuracy_score(y_test,rf_bow.predict(X_test_transform))

0.6416968591789236

In [ ]:
rf_wv = RandomForestClassifier(n_estimators=100,max_depth=5,random_state=RANDOM_SEED).fit(X_train_wv,y_train)

In [ ]:
accuracy_score(y_test,rf_wv.predict(X_test_wv))

# 3. Unsupervised Learning

In [ ]:
kmeans = KMeans(n_clusters=2,random_state=RANDOM_SEED).fit(X_train_transform)

In [ ]:
cluster_df = pd.DataFrame({'cluster':kmeans.labels_,'y_label':y_train,'text':X_train})
cluster_df

### LDA Topic Modeling - Consider NMF to create a document-topic matrix

In [267]:
import gensim
from nltk.stem.porter import *
def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    #Un-hash next line to use stemming
    #return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
    #Un-hash next line to NOT use stemming
    return WordNetLemmatizer().lemmatize(text, pos='v')
# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            #Un-hash next line to use stemming
            result.append(lemmatize_stemming(token))
            #Un-hash next line to NOT use stemming
            #result.append(token)
            
    return result

In [268]:
df['original_text'][0]

"There is manuscript evidence that Austen continued to work on these pieces as late as the period 1809 Ã¢ '' 11 , and that her niece and nephew , Anna and James Edward Austen , made further additions as late as 1814 ."

In [269]:
preprocess(df['original_text'][0])

['manuscript',
 'evidence',
 'austen',
 'continue',
 'work',
 'piece',
 'late',
 'period',
 'niece',
 'nephew',
 'anna',
 'jam',
 'edward',
 'austen',
 'additions',
 'late']

In [238]:
# This cell will run about 2 minutes
processed_docs = [preprocess(text) for text in df['original_text']]

In [239]:
dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary

In [240]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
#bow_corpus

In [241]:
len(bow_corpus)

416768

In [242]:
# This cell will run 10 minutes
#lda_model =  gensim.models.LdaMulticore(bow_corpus, 
#                                   num_topics = 8, 
#                                   id2word = dictionary,                                    
#                                   passes = 10,
#                                   workers = 2)

In [347]:
#for idx, topic in lda_model.print_topics(-1):
#    print("Topic: {} \nWords: {}".format(idx, topic ))
#    print("\n")